<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/RAG_lessons/blob/main/RAG_lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RAG pipeline cơ bản (với LlamaIndex và Llama 3)**

Đây là notebook cho các bạn để có thể tìm hiểu về cách tạo ra một mô hình RAG đơn giản, với LlamaIndex và open-source LLM như Llama 3.

Lời cảm ơn đặc biệt dành cho Plaban Nayak: [Build an Advanced Reranking-RAG System Using Llama-Index, Llama 3 and Qdrant](https://medium.com/@nayakpplaban/build-an-advanced-reranking-rag-system-using-llama-index-llama-3-and-qdrant-a8b8654174bc), vì ý tưởng cho notebook này.

***Cài đặt các thư viện cần thiết:***

Trước tiên, việc cần làm đầu tiên là cài đặt các thư viện cần thiết cho RAG pipeline. Một số điều cần lưu ý:
- LlamaIndex là framework chính chúng ta sẽ sử dụng, nó có hỗ trợ cho người dùng với kết nối LLM và các mô hình cho embeddings.
- Embeddings và LLM ở đây đều là mô hình mã nguồn mở, nên các mô hình này sẽ cần HuggingFace tokens (chìa khóa để kết nối với HuggingFace).
- sentence-transformers là phiên bản riêng để kích hoạt thư viện transformers cho xử lý câu trong NLP.

In [ ]:
%%writefile requirements.txt
llama-index
llama-index-llms-huggingface
llama-index-embeddings-fastembed
bitsandbytes
llama-index-embeddings-huggingface
fastembed
deepeval
einops
accelerate
sentence-transformers

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.0/257.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.6 MB/s eta 0:00:00
INFO: pip is looking at 

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

***Nhập tài liệu (Loading):***

Tiếp theo đó, các bạn cần nhập tài liệu vào trong vector database.

Ở notebook này, tài liệu chúng ta lựa chọn là một trích đoạn trong bản dịch tiếng Việt của "Trận chiến cuối cùng", một tác phẩm về trận Berlin trong thế chiến II: [Wikipedia](https://en.wikipedia.org/wiki/The_Last_Battle_(Ryan_book)).

Tài liệu sẽ được đưa vào VectorStoreIndex (vector database cơ bản nhất của LlamaIndex), để lưu trữ cho query của các bạn.

In [1]:
!wget "https://raw.githubusercontent.com/hoangcuongnguyen2001/RAG_lessons/main/Last_battle.txt" -O Last_battle.txt

--2024-07-24 13:17:33--  https://raw.githubusercontent.com/hoangcuongnguyen2001/RAG_lessons/main/Last_battle.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11281 (11K) [text/plain]
Saving to: ‘Last_battle.txt’

Last_battle.txt     100%[===================>]  11.02K  --.-KB/s    in 0s      

2024-07-24 13:17:33 (78.6 MB/s) - ‘Last_battle.txt’ saved [11281/11281]



In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["Last_battle.txt"]
).load_data()

Tiếp theo đó, các bạn có thể cài đặt system prompt (prompt hướng dẫn cho LLM nên làm gì), cùng với query wrapper prompt (để quy định cấu trúc cho query của người hỏi trước khi nhập vào LLM). Lưu ý, query wrapper prompt của mỗi LLM có thể tìm được trên model card tại HuggingFace; và trong ví dụ này mình sử dụng wrapper prompt từ Llama-3-8B-Instruct model card. Argument này có thể không cần thiết với các model khác.

Một điều đáng lưu ý nữa là nếu không có system prompt, hành vi ngầm định khi hỏi Llama3 bằng tiếng Việt là nó có thể thêm tiếng Anh trong câu trả lời.

In [ ]:

from llama_index.core import PromptTemplate
system_prompt = "Bạn là một trợ lý AI đắc lực. Hãy trả lời các câu hỏi càng chính xác càng tốt. Lưu ý là bạn chỉ trả lời bằng tiếng Việt và không dùng ngôn ngữ nào khác."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

***Indexing và lưu trữ tài liệu:***

Sau khi nhập tài liệu vào database, việc tiếp theo sẽ là tạo embedding cho từng phần nhỏ một (chunk), trước khi nhập embedding vào vector store index (đã được kích hoạt trước đó).

Lưu ý: embedding model chúng ta cần dùng nên là một mô hình hỗ trợ embedding cho nhiều ngôn ngữ (ở đây là *multilingual-e5-base*), hoặc một mô hình được huấn luyện với tài liệu của ngôn ngữ chúng ta muốn dùng trong câu hỏi. Với công nghệ hiện tại, chúng ta chưa thể chọn một mô hình embedding bất kỳ cho một LLM bất kỳ được.

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base", trust_remote_code=True)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Tiếp theo đó, tokenizer và LLM sẽ được tải lên cùng nhau (ở đây cả tokenizer và LLM đều từ chung một mô hình).

Stoppings_ids ở đây được sử dụng để giúp cho một LLM có thể hiểu được về việc kết thúc của từng câu là gì (nó tùy thuộc vào cấu trúc của từng LLM, như trong notebook này thì stopping_ids có thể tìm trong model card).

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token,
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


*Lưu ý khi nhập và setup LLM:*

1. Do môi trường làm việc của chúng ta là bản free của Google Colab (chỉ có 15GB RAM), các bạn nên sử dụng quantization (4 hoặc 8 bit) tùy thuộc vào kích cỡ LLM, để có thể nhập được model vào server.

2. Các bạn có thể setup temperature trong ngưỡng từ 0 tới 1, nên để temperature ở mức gần 0 nhất có thể để đảm bảo cho kết quả được chính xác.

3. Context windowm, chunk size và max new tokens ở đây đều là cài đặt ngầm định của Llama3-8B-Instruct.

In [ ]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    context_window=8192,
    max_new_tokens=256,
    model_kwargs={
        "token": hf_token,
        "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.1,
        "top_p": 0.3,

    },
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from llama_index.core import Settings


Settings.embed_model = embed_model

Settings.chunk_size = 512
# Llama-3-8B-Instruct model
Settings.llm = llm

Tiếp theo, embedding sẽ được lưu trữ vào VectorStoreIndex (với LlamaIndex).

In [ ]:
index = VectorStoreIndex.from_documents(documents)

***Đặt câu hỏi cho LLM (querying):***

Bước cuối cùng là đặt câu hỏi cho LLM thông qua vector database (như trong ví dụ này, chúng ta có thể đặt câu hỏi thông qua VectorStoreIndex (as_query_engine).

Lưu ý: Chúng ta có thể điều chỉnh similarity_top_k (top bao nhiêu từ mà LLM đoán là có khả năng nối tiếp từ cũ). Thông thường, similarity_top_k càng thấp thì kết quả đưa ra càng chính xác.

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3, streaming=True)
#query_engine = index.as_query_engine(similarity_top_k=2)

In [ ]:
response = query_engine.query("Chuyện gì đã xảy ra với Sở thú Berlin?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Câu trả lời có thể được in trực tiếp (dùng *print()*), hoặc in theo từng từ một, giống như cách ChatGPT hoặc các LLM khác đưa ra câu trả lời (dùng *print_response_stream()*).

In [ ]:
#response.print_response_stream()
print(response)

Sở thú Berlin đã bị hư hại nghiêm trọng. Các khu nuôi bò sát, hà mã, kangaroo, hổ và voi đã bị hư hại nghiêm trọng, cùng với các tòa nhà bị thủng toang hoác khác. Hồ cá đã bị hủy hoại hoàn toàn.
